In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/project leo/data2"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")
box_scores = os.listdir(SCORES_DIR)
box_scores = [os.path.join(SCORES_DIR, f) for f in box_scores if f.endswith(".html")]

In [ ]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [ ]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [ ]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [ ]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [ ]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 9662
200 / 9662
300 / 9662
400 / 9662
500 / 9662
600 / 9662
700 / 9662
800 / 9662
900 / 9662
1000 / 9662
1100 / 9662
1200 / 9662
1300 / 9662
1400 / 9662
1500 / 9662
1600 / 9662
1700 / 9662
1800 / 9662
1900 / 9662
2000 / 9662
2100 / 9662
2200 / 9662
2300 / 9662
2400 / 9662
2500 / 9662
2600 / 9662
2700 / 9662
2800 / 9662
2900 / 9662
3000 / 9662
3100 / 9662
3200 / 9662
3300 / 9662
3400 / 9662
3500 / 9662
3600 / 9662
3700 / 9662
3800 / 9662
3900 / 9662
4000 / 9662
4100 / 9662
4200 / 9662
4300 / 9662
4400 / 9662
4500 / 9662
4600 / 9662
4700 / 9662
4800 / 9662
4900 / 9662


KeyboardInterrupt: ignored

In [ ]:
games_df.to_csv('parsed1.csv')